In [2]:
# Sam Brown
# Sam_brown@mines.edu
# June 20
# Goal: Use LSTM neural nets to capture and leverage long and short term patterns in the tidal modulation

import sys
sys.path.append("/Users/sambrown04/Documents/SURF/whillans-surf/notebooks/SURF")

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split, TensorDataset
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

df = pd.read_csv("/Users/sambrown04/Documents/SURF/Preproc_data/10-18.csv", parse_dates=["start_time"])
df = df.iloc[500:3000] # Account for gaps in time-series data

In [4]:
# Features and Target
X = df[['tide_height', 'tide_deriv', 'A_diurn', 'A_semidiurn', 'high_t_evt', 'time_since', 'sev_stds', 'pre-s_stds', 'form_fac']]
y = df['slip_size_standardized'].values.reshape(-1,1)

# Normalize features
x_scaler = StandardScaler()
X_scaled = x_scaler.fit_transform(X)

y_scaler = StandardScaler()
y_scaled = y_scaler.fit_transform(y)

In [6]:
SEQ_LEN =  30 # Sequence of "memory"
batch_size = 32

sequences = []
targets = []

# We Want to create rolling sequences of SEQ_LEN
for i in range(len(X_scaled) - SEQ_LEN):
    seq = X_scaled[i:i+SEQ_LEN]
    target = y_scaled[i + SEQ_LEN - 1][0]  # flatten from 2D array NOTE: To change which target we predict remove -1 -> predicts target after last row of features.
    sequences.append(seq)
    targets.append(target)

#Tensors
X_seq = torch.tensor(np.array(sequences), dtype=torch.float32)
y_seq = torch.tensor(np.array(targets), dtype=torch.float32).unsqueeze(1)

print("X_seq shape:", X_seq.shape)  # [num_samples, seq_len, 6]
print("y_seq shape:", y_seq.shape)  # [num_samples, 1]

X_seq shape: torch.Size([2470, 30, 9])
y_seq shape: torch.Size([2470, 1])


In [8]:
# Train test to prevent data leakage
train_size = int(0.8 * len(X_seq))

X_train = X_seq[:train_size]
y_train = y_seq[:train_size]
X_test = X_seq[train_size:]
y_test = y_seq[train_size:]

# Create DataLoaders with no shuffling
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=False)
test_loader  = DataLoader(TensorDataset(X_test, y_test), batch_size=32, shuffle=False)

In [9]:
# Model
class SlipLSTM(nn.Module):
    def __init__(self, input_size=9, hidden_size=64, num_layers=1):
        super(SlipLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size,
                            hidden_size=hidden_size,
                            num_layers=num_layers,
                            batch_first=True)

        self.fc1 = nn.Linear(hidden_size, 32)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.3) # Dropout, saw on forum... prevents overfitting
        self.fc2 = nn.Linear(32, 1)

    def forward(self, x):
        out, _ = self.lstm(x)       # out: batch, seq_len, hidden_size
        out = out[:, -1, :]         # take output at last time step
        out = self.fc1(out)         # linear layer 1
        out = self.relu(out)        # ReLU activation
        out = self.fc2(out)         # final output layer
        return out

In [10]:
model = SlipLSTM(input_size=9, hidden_size=64, num_layers=1)

# Loss and optimizer
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [11]:
num_epochs = 30

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for batch_X, batch_y in train_loader: # mini- batch loop
        optimizer.zero_grad()
        preds = model(batch_X)
        loss = loss_fn(preds, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")

Epoch 1/30, Loss: 0.6474
Epoch 2/30, Loss: 0.1781
Epoch 3/30, Loss: 0.1264
Epoch 4/30, Loss: 0.1011
Epoch 5/30, Loss: 0.0836
Epoch 6/30, Loss: 0.0726
Epoch 7/30, Loss: 0.0658
Epoch 8/30, Loss: 0.0605
Epoch 9/30, Loss: 0.0574
Epoch 10/30, Loss: 0.0563
Epoch 11/30, Loss: 0.0561
Epoch 12/30, Loss: 0.0537
Epoch 13/30, Loss: 0.0515
Epoch 14/30, Loss: 0.0505
Epoch 15/30, Loss: 0.0497
Epoch 16/30, Loss: 0.0490
Epoch 17/30, Loss: 0.0486
Epoch 18/30, Loss: 0.0482
Epoch 19/30, Loss: 0.0478
Epoch 20/30, Loss: 0.0471
Epoch 21/30, Loss: 0.0470
Epoch 22/30, Loss: 0.0466
Epoch 23/30, Loss: 0.0463
Epoch 24/30, Loss: 0.0458
Epoch 25/30, Loss: 0.0452
Epoch 26/30, Loss: 0.0436
Epoch 27/30, Loss: 0.0429
Epoch 28/30, Loss: 0.0429
Epoch 29/30, Loss: 0.0413
Epoch 30/30, Loss: 0.0397


In [12]:
model.eval()

preds, trues = [], []

with torch.no_grad():
    for batch_X, batch_y in test_loader:
        pred = model(batch_X)
        preds.append(pred.numpy())
        trues.append(batch_y.numpy())

preds = np.vstack(preds)
trues = np.vstack(trues)

mse = mean_squared_error(trues, preds)
print(f"Test MSE: {mse:.4f}")

Test MSE: 0.4846


In [13]:
r2 = r2_score(trues, preds)
print("R² score:", r2)

R² score: 0.522799015045166
